#### Installing necessary libs

In [7]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame

#### Importing OpenAI gym and other dependencies and testing environments

In [5]:
import random
import gym
import os
os.environ['SDL_VIDEODRIVER']='dummy'

#### Declaring environment and creating states

In [6]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
print("number os states:", states)
actions = env.action_space.n
print("number os actions:", actions)

number os states: 4
number os actions: 2


#### Creating Iteration with n episodes and passing the action

In [7]:
episodes = 10
for episode in range(1, episodes+10):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))

Episode:1 Score:28.0
Episode:2 Score:14.0
Episode:3 Score:22.0
Episode:4 Score:39.0
Episode:5 Score:54.0
Episode:6 Score:32.0
Episode:7 Score:19.0
Episode:8 Score:22.0
Episode:9 Score:12.0
Episode:10 Score:16.0
Episode:11 Score:24.0
Episode:12 Score:10.0
Episode:13 Score:15.0
Episode:14 Score:26.0
Episode:15 Score:27.0
Episode:16 Score:20.0
Episode:17 Score:20.0
Episode:18 Score:12.0
Episode:19 Score:33.0


#### Modelling with Keras

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [14]:
import tensorflow

def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(units=24, activation='relu' ))
    model.add(Dense(units=24, activation='relu'))
    model.add(Dense(units=actions, activation='linear'))
    return model

In [15]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


#### Initialising Agent with Keras - Reinforced Learning

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 22:01 - reward: 1.0000

/Users/tejaspatel/IdeaProjects/AI-class-practice/venv/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 103s 10ms/step - reward: 1.0000
107 episodes - episode_reward: 92.925 [10.000, 200.000] - loss: 2.758 - mae: 18.731 - mean_q: 37.939

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 93s 9ms/step - reward: 1.0000
51 episodes - episode_reward: 194.137 [171.000, 200.000] - loss: 5.421 - mae: 37.890 - mean_q: 76.689

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 95s 10ms/step - reward: 1.0000
51 episodes - episode_reward: 198.059 [174.000, 200.000] - loss: 5.478 - mae: 40.641 - mean_q: 82.015

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.0000
51 episodes - episode_reward: 195.314 [148.000, 200.000] - loss: 4.532 - mae: 38.766 - mean_q: 77.992

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: 1.0000
done, took 461.556 seconds


In [19]:
scores = dqn.test(env, nb_episodes=15, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 192.000, steps: 192
Episode 5: reward: 187.000, steps: 187
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 195.000, steps: 195
Episode 9: reward: 183.000, steps: 183
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 176.000, steps: 176
Episode 12: reward: 199.000, steps: 199
Episode 13: reward: 183.000, steps: 183
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
194.33333333333334


In [20]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 196.000, steps: 196


##### references
##### 1. https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic
##### 2. https://gsurma.medium.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288
##### 3. https://github.com/ChaithanyaVamshi/CartPole-DQN-Reinforcementlearning
##### 4. https://github.com/alishafique3/CartPole-DQN-model-in-Google-Colab/blob/main/RL_Classical_Framework.ipynb
##### 5. https://stackoverflow.com/questions/tagged/dqn